










<img style="float:left" width="70%" src="pics/escudo_COLOR_1L_DCHA.png">
<br style="clear:both;">


# *Linja*
### *Sistemas Inteligentes* (Curso 2023-2024)


## Docentes

 - Pedro Latorre Carmona

## Estudiante / Autor de la práctica

 - Ignacio Zaldo González

## Código del programa


#### Lo primero que hago es la parte del código sin el miniMax, es decir, toda la parte que permite jugar de manera "manual" al juego.

- Importo las librerias que voy a usar en el código:

In [1]:
from copy import deepcopy
from typing import Tuple, List
from IPython.display import HTML, display
from ipywidgets import HTML
import math

- Código con la lógica y los métodos necesarios para el funcionamiento del juego.

In [2]:
class Tablerolinja:
    
    def __init__(self, matrix):
        self.setMatrix(matrix)    # Creo todas las variables globales, o variables que voy a usar en todo el código
        self.turno = 1            # Para el turno
        self.movimiento = 0       # Para cambiar entre movimientos (1 o 2)
        self.fichasFila = 0       # Para almacenar el número de fichas por fila
        self.row_nueva = 0        # Para almacenar el valor de la fila nueva
        self.turnoExtra = False   # Para saber si hay turno extra o no
    
    def __eq__(self, other) -> bool:  
        return self.getMatrix() == other.getMatrix()    # Comparo si una matriz es igual a otra diferente
    
    def __iter__(self):
        return iter(self.matrix)  # Devuelvo un iterador sobre la matriz, para permitir que el tablero sea iterable
    
    def setMatrix(self, matrix):
        self.matrix = deepcopy(matrix)   # Creo una copia de la matriz para igualarla a self.matrix
    
    def getMatrix(self) -> List[List]:
        return deepcopy(self.matrix)     # Devuelvo una copia de la matriz
    
    
    def placeTile(self, row: int, col: int, tile: int):
        if 0 <= row < len(self.matrix) and 0 <= col < len(self.matrix[0]):  # Si la fila está entra 0 y la longitud del tablero y la columna también
            self.matrix[row][col] = tile                                    # Pongo en esa posición de fila y columna la ficha correspondiente
            return 1  # Indica que la ficha fue colocada con éxito
        else:
            raise ValueError("No hay ficha en esa posición o estás eligiendo una fuera de los límites del tablero")   # En caso de fallo, lanzo este error
    

    def deleteTile(self, row: int, col: int):
        if 0 <= row < len(self.matrix) and 0 <= col < len(self.matrix[0]):  # Si la fila está entra 0 y la longitud del tablero y la columna también
            self.matrix[row][col] = 0                                       # Pongo en esa posición de fila y columna el valor 0, quitando así la ficha
            return 1  # Indica que la ficha fue eliminada con éxito
        else:
            raise ValueError("No hay ficha en esa posición o estás eligiendo una fuera de los límites del tablero")   # En caso de fallo, lanzo este error

    
    def utility(self) -> int:
        valores_filas = [5, 3, 2, 1, -1, -2, -3, -5]    # Puntuación asignada a cada una de las filas del tablero
        total_utility_rojo = 0                          # Variables para conseguir la puntuación de cada jugador
        total_utility_negro = 0
        
        mitad_tablero = len(self.matrix) // 2           # Para obtener la mitad del tablero

        for row in range(len(self.matrix)):                     # Recorro las filas
            for col in range(len(self.matrix[0])):              # Recorro las columnas
                ficha = self.matrix[row][col]                   # Guardo en ficha el valor de esa fila y columna
                if ficha == 1:                                          # Si el valor es 1 (ficha jugador negro)
                    if row < mitad_tablero:                             # Calculo de esta manera los valores
                        total_utility_rojo -= valores_filas[row]
                    else:
                        total_utility_rojo += valores_filas[row]
                elif ficha == 2:                                        # Si el valor es 2 (ficha jugador rojo)
                    if row >= mitad_tablero:                            # Calculo de esta otra manera los valores
                        total_utility_negro -= valores_filas[row]
                    else:
                        total_utility_negro += valores_filas[row]            
        return total_utility_rojo - total_utility_negro        # Devuelvo la diferencia entre las puntuaciones de los jugadores
    
    
    def numFichasEnFila(self, row ) -> int:        # Esta función lo que hace es calcular el número de fichas que tiene una fila, para así calcular el segundo movimiento de los jugadores correctamente
        numFichas = 0                    # Creo la variable
        fila = self.matrix[row]          # Guardo la fila
        for i in fila:                   # Recorro las filas
            if i != 0:                   # Si hay ficha
                numFichas += 1           # Sumo 1 al valor de la variable que había creado
        return numFichas                 # Devuelvo el número de fichas de la fila pasada por parámetro
    

    def cambiarTurno(self):              # En esta función, lo que hago es cambiar el turno de los jugadores
        if self.turno == 1:              # Si el turno es 1 cambio al turno 2
            self.turno = 2
        elif self.turno == 2:            # Si el turno es 2 cambio al turno 1
            self.turno = 1


    def comprobarPosicion(self, row, col):              # Esta función la utilizo para ver si la posición de la ficha y la columna escogida está dentro del tablero
        if row < 0 or row >= len(self.matrix) or col < 0 or col >= len(self.matrix[0]):           # Si la posición no está dentro del tablero devuelvo False
            return False
        
        if self.matrix[row][col] == 0 or self.matrix[row][col] == self.turno:                     # Verifico si la posición está vacía o tiene una ficha del mismo color y devuelvo True
            return True
        
        if (row == 0 and self.matrix[row][col] == 2) or (row == len(self.matrix) - 1 and self.matrix[row][col] == 1):  # Verifico si la posición está en el extremo opuesto y tiene una ficha del color contrario
            if self.matrix[row][col] == 1 - self.turno:
                return True

        return False         # En cualquier otro caso, la posición no es válida
    
    
    def moverFicha(self, row, col):        # Esta función es la que utilizo para el movimiento de las fichas
        ficha = self.matrix[row][col]      # Obtengo la ficha en la posición actual
        
        if ficha != 0 and ficha != self.turno:            # Verifico que la ficha en la posición actual coincide con el turno actual
            print("No puedes mover una ficha del otro jugador.")

        # Llamo al método correspondiente según el tipo de movimiento
        if self.movimiento == 0:         # Si el valor de movimiento es 0
            resultado_movimiento = self.movimiento1(row, col)           # Llamo al primer movimiento y devuelvo el resultado
        elif self.movimiento == 1:       # Si el valor de movimiento es 0
            resultado_movimiento = self.movimiento2(row, col)           # Llamo al segundo movimiento y devuelvo el resultado
        
        if self.finJuego():     # Si el juego finaliza
            print("El juego ha terminado, no mueva más fichas")   # Imprimo la advertencia por pantalla
            self.endGame()                                        # Llamo a endGame
        return resultado_movimiento                   # Devuelvo resultado del movimiento correspondiente
    
    
    def movimiento1(self, row, col):           # En esta función hago el primer movimiento de los jugadores
        ficha = self.matrix[row][col]          # Almaceno el valor de la ficha en la posición seleccionada
        
        # Calculo la nueva fila a la que se moverá la ficha
        if(row + 1 <= 7 and ficha == 1):        
            self.row_nueva = row + 1
        elif(row - 1 >= 0  and ficha == 2):
            self.row_nueva = row - 1
        elif(row + 1 > 7 and ficha == 1):
            self.row_nueva = 7
        elif(row - 1 < 0 and ficha == 2):
            self.row_nueva = 0
        else:
            return 0
        
        # Verifico si el jugador no alcanzó el extremo opuesto
        if 0 <= self.row_nueva < len(self.matrix):
            col_nueva = col
            # Verifico si hay ficha en la fila de destino
            if self.matrix[self.row_nueva][col] != 0:
                col_candidata = col_nueva
                # Creo una variable booleana que indique si se ha encontrado una columna vacía
                encontrado = False
                i = 1  # Comienzo con el primer desplazamiento
                # Si hay ficha en la columna destino, desplazo lateralmente hacia la derecha o izquierda
                while i <= 5 and not encontrado:
                    col_candidata = col_nueva + i
                    # Verifico si la columna candidata está dentro del rango válido
                    if 0 <= col_candidata < len(self.matrix[0]):
                        # Verifico si la columna candidata está vacía
                        if self.matrix[self.row_nueva][col_candidata] == 0:
                            # Asigno la columna candidata a la variable col_nueva
                            col_nueva = col_candidata
                            # Cambio el valor de la variable encontrado a True
                            encontrado = True
                            # Rompo el bucle cuando se encuentre una columna vacía
                            break
                    i += 1
                i = 1  # Restablezco el valor de i para el desplazamiento hacia la izquierda
                while i <= 5 and not encontrado:
                    col_candidata = col_nueva - i
                    # Verifico si la columna candidata está dentro del rango válido
                    if 0 <= col_candidata < len(self.matrix[0]):
                        # Verifico si la columna candidata está vacía
                        if self.matrix[self.row_nueva][col_candidata] == 0:
                            # Asigno la columna candidata a la variable col_nueva
                            col_nueva = col_candidata
                            # Cambio el valor de la variable encontrado a True
                            encontrado = True
                            # Rompo el bucle cuando se encuentre una columna vacía
                            break
                    i += 1  
                # Verifico si la nueva posición es válida
                if self.comprobarPosicion(self.row_nueva, col_nueva):
                    self.fichasFila = self.numFichasEnFila(self.row_nueva)
                    if self.numFichasEnFila(self.row_nueva) > 0:
                        # Cambio el valor del atributo movimiento a 1, indicando que se puede hacer el segundo movimiento
                        self.movimiento = 1           
                    elif self.numFichasEnFila(self.row_nueva) == 0:
                        self.cambiarTurno()    
                    # Muevo la ficha
                    self.deleteTile(row, col)
                    self.placeTile(self.row_nueva, col_nueva, ficha)
                    return 1
            else:
                # Verifico si la nueva posición es válida
                if self.comprobarPosicion(self.row_nueva, col):
                    self.fichasFila = self.numFichasEnFila(self.row_nueva)
                    if self.numFichasEnFila(self.row_nueva) > 0:
                        # Cambio el valor del atributo movimiento a 1, indicando que se puede hacer el segundo movimiento
                        self.movimiento = 1           
                    elif self.numFichasEnFila(self.row_nueva) == 0:
                        self.cambiarTurno() 
                    # Muevo la ficha
                    self.deleteTile(row, col)
                    self.placeTile(self.row_nueva, col, ficha)
                    return 1
        return 0
    
    
    def movimiento2(self, row, col):                # En esta función hago el segundo movimiento de los jugadores
        ficha = self.matrix[row][col]               # Almaceno el valor de la ficha en la posición seleccionada
        # Uso el valor de row que se ha pasado como parámetro
        row_nueva = row
        # Creo una variable booleana que indica si el jugador ha llegado al extremo opuesto
        extremo_opuesto = False
        if self.comprobarPosicion(row_nueva, col):
            # Obtengo el número de fichas en la fila a la que se movió la ficha en el primer movimiento
            num_fichas_en_fila = self.fichasFila 
            if num_fichas_en_fila == 0: 
                row_nueva = row
            else:                                   # Compruebo las posiciones de fila nueva posibles para todos los casos
                if(row + num_fichas_en_fila <= 7 and ficha == 1):
                    row_nueva = row + num_fichas_en_fila
                elif(row - num_fichas_en_fila >= 0  and ficha == 2):
                    row_nueva = row - num_fichas_en_fila
                elif(row + num_fichas_en_fila > 7 and ficha == 1):
                    row_nueva = 7
                elif(row - num_fichas_en_fila < 0 and ficha == 2):
                    row_nueva = 0
                else:
                    return 0
            col_primera = col
            if row_nueva == 7 and ficha == 1:  # Si la fila nueva es la 7 y la ficha 1, hago el movimiento y pongo que he llegado al extremo opuesto
                if self.matrix[row][col] != 0:
                    col = 0
                self.deleteTile(row, col_primera)
                self.placeTile(row_nueva, col, ficha) 
                extremo_opuesto = True
            elif row_nueva == 0 and ficha == 2: # Si la fila nueva es la 0 y la ficha 2, hago el movimiento y pongo que he llegado al extremo opuesto
                if self.matrix[row][col] != 0:
                    col = 0
                self.deleteTile(row, col_primera)
                self.placeTile(row_nueva, col, ficha) 
                extremo_opuesto = True    
            col_nueva = col
            # Verifico si el jugador no alcanzó el extremo opuesto
            if row_nueva not in [0, len(self.matrix)] and not extremo_opuesto:
                # Verifico si hay ficha en la fila de destino
                if self.matrix[row_nueva][col] != 0:
                    col_candidata = col_nueva
                    # Si hay ficha, desplazo lateralmente hacia la derecha o izquierda
                    # Creo una variable booleana que indique si se ha encontrado una columna vacía
                    encontrado = False
                    i = 1  # Empiezo con el primer desplazamiento
                    while i <= 5 and not encontrado:
                        col_candidata = col_nueva + i
                        # Verifico si la columna candidata está dentro del rango válido
                        if 0 <= col_candidata < len(self.matrix[0]):
                            # Verificar si la columna candidata está vacía
                            if self.matrix[row_nueva][col_candidata] == 0:
                                # Asigno la columna candidata a la variable col_nueva
                                col_nueva = col_candidata
                                # Cambio el valor de la variable encontrado a True
                                encontrado = True
                                # Rompo el bucle cuando se encuentre una columna vacía
                                break
                        i += 1
                    i = 1  # Restablezco el valor de i para el desplazamiento hacia la izquierda
                    while i <= 5 and not encontrado:
                        col_candidata = col_nueva - i
                        # Verifico si la columna candidata está dentro del rango válido
                        if 0 <= col_candidata < len(self.matrix[0]):
                            # Verifico si la columna candidata está vacía
                            if self.matrix[row_nueva][col_candidata] == 0:
                                # Asigno la columna candidata a la variable col_nueva
                                col_nueva = col_candidata
                                # Cambio el valor de la variable encontrado a True
                                encontrado = True
                                # Rompo el bucle cuando se encuentre una columna vacía
                                break
                        i += 1
                    # Verifico si la nueva posición es válida
                    if self.comprobarPosicion(row_nueva, col_nueva):
                        self.fichasFila = self.numFichasEnFila(row_nueva)   
                        # Muevo la ficha
                        self.deleteTile(row, col)
                        self.placeTile(row_nueva, col_nueva, ficha)
                else:
                    # Verifico si la nueva posición es válida
                    if self.comprobarPosicion(row_nueva, col):
                        self.fichasFila = self.numFichasEnFila(row_nueva)        
                        # Muevo la ficha
                        self.deleteTile(row, col)
                        self.placeTile(row_nueva, col, ficha)
            # Verifico si la ficha no cae en una fila vacía
            if self.numFichasEnFila(row_nueva) - 1 > 0:
                # Cambio el valor del atributo movimiento a 1, indicando que se puede hacer el segundo movimiento
                self.movimiento = 0
                if self.turnoExtra == 1:
                    self.turnoExtra = False                                             
            elif self.numFichasEnFila(row_nueva) - 1 == 0:
                self.movimiento = 0
                self.turnoExtra = True
            if self.turnoExtra == False:
                self.cambiarTurno()  
            return 1
        else:
            return 0

    
    def finJuego(self):     # En esta función veo si se han sobrepasado las fichas de los 2 jugadores para ver si ha terminado el juego
        # Inicializo las variables que guardan la posición de la ficha más atrasada de cada tipo de ficha
        fila_2 = -1
        col_2 = -1
        fila_1 = len (self.matrix) 
        col_1 = len (self.matrix[0])

        # Recorro el tablero de arriba a abajo para buscar la ficha más atrasada de las fichas 2
        for i in range (len (self.matrix)):
            # Si hay alguna ficha 2 en la fila i, uso index (2) para obtener su columna
            if 2 in self.matrix [i]:
                j = self.matrix [i].index (2)
                # Si la fila i es mayor que la fila guardada, o si la fila es la misma pero la columna es menor, actualizo la posición de la ficha más atrasada
                if i > fila_2 or (i == fila_2 and j < col_2):
                    fila_2 = i
                    col_2 = j
        # Recorro el tablero de abajo a arriba para buscar la ficha más atrasada de las fichas 1
        for i in range (len (self.matrix) - 1, -1, -1):
            # Si hay alguna ficha 1 en la fila i, uso index (1) para obtener su columna
            if 1 in self.matrix [i]:
                j = self.matrix [i].index (1)
                # Si la fila i es menor que la fila guardada, o si la fila es la misma pero la columna es menor, actualizo la posición de la ficha más atrasada
                if i < fila_1 or (i == fila_1 and j < col_1):
                    fila_1 = i
                    col_1 = j
        
        if fila_2 < fila_1:
            return True
        return False
        

    def endGame(self) -> int:   # Función para determinar qué jugador es el ganador
        utilidad = self.utility()
        
        if utilidad > 0:
            return 1  # Rojo gana
        elif utilidad < 0:
            return 2  # Negro gana
        else:
            return 0  # Empate
    
    
    def get_content(self,row,col):   # Esta función la utilizo para asignar el número de ficha a un tipo de imagen, para la visualización del tablero HTML
        
        contenido = [None]

        if self.matrix[row][col] == 0:
            contenido[0] = "casillavacia"
        elif self.matrix[row][col]  == 1:
            contenido[0] = "casillanegra"
        elif self.matrix[row][col]  == 2:
            contenido[0] = "casillaroja"
                
        return contenido
    
    
    def get_html(self):            # Esta función sirve para representar el tablero del juego
        element_image = {          # Elijo las imagenes que voy a usar
            "casillavacia": "./ImagenesCasillasLinja/CasillaVacia.png",
            "casillanegra": "./ImagenesCasillasLinja/CasillaNegra.png",
            "casillaroja": "./ImagenesCasillasLinja/casillaRoja.png"
        }
        height = len(self.matrix)
        width = len(self.matrix[0])

        html_string = "<style> img.game {width: 50px !important; height: 40px !important;}</style><table>"  # Elijo una altura y anchura para que se vea bien el tablero

        for i in range(height):        # Represento el tablero
            new_row = "<tr>"
            for j in range(width):
                content = self.get_content(i, j)
                drawing = element_image[content[0]]
                html = '<td><img class="game" src=%s alt=""></img></td>' % drawing
                new_row += html

            new_row += "</tr>"
            html_string += new_row

        html_string += "</table>"
        return html_string
        
    
    def moveCanBeMade(self, player: int) -> bool:       # Esta función la utilizo después en miniMax y sirve para ver si el movimiento se puede hacer o no
        for row in range(len(self.matrix)):             # Recorro filas y columnas
            for col in range(len(self.matrix[0])):
                if self.comprobarPosicion(row, col) and self.matrix[row][col] == player:            # Si la posición es valida devuelvo true
                    return True
        return False
    
    
    def generateSuccessors(self, player):               # Esta función la utilizo después en miniMax y sirve para obtener los sucesores
        successors = []
        for row in range(len(self.matrix)):             # Recorro filas y columnas
            for col in range(len(self.matrix[0])):
                successorBoard_ = Tablerolinja(self.matrix)             # Hago una copia del tablero original
                if successorBoard_.comprobarPosicion(row, col):         # Si la posición es valida
                    successorBoard_.moverFicha(row,col)                 # Muevo la ficha  
                    for row_iter in range(len(self.matrix)):            # Recorro filas y columnas de nuevo
                        for col_iter in range(len(self.matrix[0])):    
                            successorBoard = deepcopy(successorBoard_)   # Copio el tablero anterior
                            if successorBoard.comprobarPosicion(row_iter, col_iter):  # Si la posición es valida
                                successorBoard.moverFicha(row_iter, col_iter)         # Muevo la ficha
                                if len(successors) == 10:  # Limito el número de sucesores para que no esté demasiado tiempo pensando el programa, porque cada vez que lo llame en miniMax generará 10 nuevos sucesores
                                    break
                                if successorBoard.turno == 2:          # Compruebo si el turno es 2 para evitar algunos errores
                                    successors.append(successorBoard)       # Relleno la lista de sucesores
            return successors 
      
        
matrix = [[1, 1, 1, 1, 1, 1],
          [1, 0, 0, 0, 0, 2],
          [1, 0, 0, 0, 0, 2],
          [1, 0, 0, 0, 0, 2],
          [1, 0, 0, 0, 0, 2],
          [1, 0, 0, 0, 0, 2],
          [1, 0, 0, 0, 0, 2],
          [2, 2, 2, 2, 2, 2]]

tablero = Tablerolinja(matrix)
html = tablero.get_html()


---
## Función Minimax (con poda $\alpha-\beta$)

##### El código es similar al proporcionado desde un principio, solo he modificado las funciones moveCanBeMade y generateSuccessors, las cuales he creado arriba en la parte del código principal.
##### Además, he cambiado los mat, dejandolos simplemente como mat = successorMatrices[i] para que no diera errores a la hora de ejecutar.
---

In [3]:

def miniMax(state:Tablerolinja, currentLevel:int, maxLevel:int, player:int, alpha:int, beta:int, stop:bool) -> Tuple[Tablerolinja, int, bool]:

    matriz=state.getMatrix() # Obtengo la matriz del estado actual
    
    successorMatrices = []  # Defino las matrices de sucesores
    
    if (not state.moveCanBeMade(player) or currentLevel == maxLevel): # Se para si no se pueden realizar más movimientos o se alcanza el nivel máximo
        return (state.matrix,state.utility(),stop)
    
    successorMatrices = state.generateSuccessors(player)
    
    # Creo el método generateSuccessors arriba, en la parte del código principal
    if len(successorMatrices) == 0:   # Si no hay sucesores, se detiene la búsqueda
        stopDigging = True
        coste=state.utility()
        return (state,coste,stopDigging)
    
    bestMatrix = None
            
    if player == 2:                    
        maxValue = -math.inf #alpha
        
        for i in range(0, len(successorMatrices)):             
                mat = successorMatrices[i]  # Cambio el Tablerolinja(successorMatrices[i]) por esto para que no de el error: object of type 'Tablerolinja' has no len()   
                # Llamada recursiva para evaluar el sucesor
                matrizS, utility, stop = miniMax(mat, currentLevel + 1, maxLevel, 1, alpha,beta,stop)
                best = utility 
                # Se actualiza el mejor movimiento
                if best > maxValue:
                    maxValue = best
                    bestMatrix = mat
                alpha = max(alpha, best)                
                # Poda beta
                if best >= beta:                 
                    return (matrizS,best,stop)     
    else:                           
        minValue = math.inf #beta
        for i in range(0, len(successorMatrices)):
                mat = successorMatrices[i]  # Cambio el Tablerolinja(successorMatrices[i]) por esto para que no de el error: object of type 'Tablerolinja' has no len()   
                # Llamada recursiva para evaluar el sucesor
                matrizS, utility, stop = miniMax(mat, currentLevel + 1, maxLevel, 2, alpha,beta,stop)
                if utility < minValue:
                    minValue = utility
                    bestMatrix = mat
                beta = min(beta, utility)
                # Poda alfa
                if utility <= alpha:                    
                   return (matrizS,utility,stop)
             
    return (bestMatrix,utility,stop) # Devuelve el mejor movimiento, su utilidad y el indicador de parada


---
### Poner en funcionamiento Minimax
---

In [4]:
def performActionMinMax(state: Tablerolinja, player: int):
    matrizB = state.getMatrix() # Se copia la matriz del estado actual
    tmpMatriz = [row[:] for row in matrizB]
    depth = 2  # Establezco la profundidad de búsqueda que voy a usar
    # Inicializo la matriz óptima y el indicador de parada
    matrizoptima = tmpMatriz
    stop = False

    # Bucle para iterar hasta 3 veces o hasta que se cumpla la condición de parada
    for itera in range(3):
        # Creo una instancia de Tablerolinja con la matriz temporal
        tmpMatrizB = Tablerolinja(tmpMatriz)
        # Llamo a la función miniMax para encontrar el mejor movimiento y valor
        (matrizoptima, valoroptimo, stop) = miniMax(tmpMatrizB, 0, depth, player, -math.inf, math.inf, stop)

    # Devuevlo tanto la matriz óptima como el valor óptimo
    return matrizoptima, valoroptimo


---
## EJECUCIÓN DEL JUEGO

##### El valor que sale como resultado del utility es el valor de las fichas rojas menos el valor de las fichas negras
---

In [5]:
def jugar(state: Tablerolinja, max_repeticiones: int = 2):
    # Inicializo el estado actual y variables de control
    resultMinMax = Tablerolinja(matrix)
    turno = resultMinMax.turno
    repeticiones = 0  # Contador de repeticiones
    estado_anterior = None  # Almaceno el estado anterior al último movimiento

    # Bucle principal del juego
    while not tablero.finJuego() and repeticiones < max_repeticiones:
        if resultMinMax.turno == 1:
            # Turno de la máquina: Guardo el estado antes de realizar el movimiento
            estado_anterior = Tablerolinja(resultMinMax.getMatrix())
            resultMinMax, valueMinMax = performActionMinMax(resultMinMax, 1)
            print("Tablero tras el movimiento de la máquina:")
            html = resultMinMax.get_html()
            display(HTML(html))
            print("Valor utility:", valueMinMax)

        elif resultMinMax.turno == 2:
            # Turno del jugador: Solicito entrada y realizar movimiento
            while True:
                try:
                    print("Por favor, especifica la fila y la columna donde está la ficha que quieres mover:")
                    fila = int(input("Fila de la ficha que se desea mover: "))
                    columna = int(input("Columna de la ficha que se desea mover: "))
                    resultMinMax.moverFicha(fila, columna)
                    break
                except ValueError:
                    print("Por favor, ingresa valores numéricos válidos.")

            html = resultMinMax.get_html()
            print("Tablero tras el movimiento de la persona:")
            display(HTML(html))

        # Verifico si el tablero es igual al estado anterior al último movimiento
        if estado_anterior is not None and resultMinMax == estado_anterior:
            repeticiones += 1
        else:
            repeticiones = 0  # Reinicio el contador si no es igual

    # Muestro el resultado final o informo sobre la detención de la ejecución por algún error en los movimientos
    if repeticiones >= max_repeticiones:
        print("Se ha detenido la ejecución, para que no se imprimir este mismo tablero infinitas veces.")
    else:
        print("Resultado final:", valueMinMax)

# Mostrar el tablero inicial
display(HTML(tablero.get_html()))
jugar(tablero)

HTML(value='<style> img.game {width: 50px !important; height: 40px !important;}</style><table><tr><td><img cla…

Tablero tras el movimiento de la máquina:


HTML(value='<style> img.game {width: 50px !important; height: 40px !important;}</style><table><tr><td><img cla…

Valor utility: -78
Por favor, especifica la fila y la columna donde está la ficha que quieres mover:
Fila de la ficha que se desea mover: 7
Columna de la ficha que se desea mover: 1
Tablero tras el movimiento de la persona:


HTML(value='<style> img.game {width: 50px !important; height: 40px !important;}</style><table><tr><td><img cla…

...

Tablero tras el movimiento de la máquina:


HTML(value='<style> img.game {width: 50px !important; height: 40px !important;}</style><table><tr><td><img cla…

Valor utility: -59
Tablero tras el movimiento de la máquina:


HTML(value='<style> img.game {width: 50px !important; height: 40px !important;}</style><table><tr><td><img cla…

Valor utility: -59
Se ha detenido la ejecución, porque sino solo se va a imprimir este mismo tablero infinitas veces hasta que se termine la ejecución manualmente.
